In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#! pip install mosestokenizer

In [ ]:
# colab　にmecabを入れる
#!apt install aptitude
#!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
#!pip install mecab-python3==0.7

In [1]:
import sys
import gc
import os
base_dir = "drive/My Drive/jp_en_translation/".replace("/",os.sep)
sys.path.append("/Users/ueki/Desktop/work/jp_en_translation")
sys.path.append("/Users/ueki/Desktop/JP_EN_Transalation")
sys.path.append(base_dir)
sys.path.append("drive/My Drive/jp_en_translation".replace("/",os.sep))

In [2]:
from models.Seq2Seq_batch_bidir import Seq2Seq
from utils.LangEn import LangEn
from utils.LangJa import LangJa
from utils.build_emb import build_en_emb,build_jp_emb
from utils.preprocess import loadLangs
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
import torch

Using TensorFlow backend.


In [3]:
#base_dir = "drive/My Drive/jp_en_translation/".replace("/",os.sep)
base_dir="../"

In [4]:
config={
    "corpus_file":base_dir+"data/jpn.txt",
    "en_col":"description_en",
    "jp_col":"description_jp",
    "SOS_token":1,
    "EOS_token":2,
    "UNK_token":0,
    "max_features":5000,
    "MAX_LENGTH":20,
    "train_size":15000,
    "val_size":100,
    "batch_size":128,
    "epochs":500,
    "maxlen_enc":20,
    "maxlen_dec":20,
    "n_hidden":300,
    "input_dim":5000,
    "output_dim":5000,
    "emb_dim":300,
    "use_enc_emb":False,
    "use_dec_emb":False,
    "validation_split":0.1,
    "trained_param_dir":base_dir+"trained_models/1_lstm_ja_en_01.hdf5",
    "translate_length":25,
    "en_W2V_FILE" : base_dir+"data/GoogleNews-vectors-negative300.bin.gz",
    "jp_W2V_FILE":base_dir+"data/ja_data/ja.bin",
    "src":"en",
    "trg":"jp",
    "learning_rate":0.01,
}

# train

In [5]:
data=loadLangs(config)

reading lines


In [6]:
val_data = data[config["train_size"]:config["train_size"]+config["val_size"]]
data = data[:config["train_size"]]

In [7]:
EN_lang = LangEn(config)
JP_lang = LangJa(config)

stdbuf was not found; communication with perl may hang due to stdio buffering.


In [8]:
for s in data[config["en_col"]]:
    EN_lang.addSentence(s)

In [9]:
for s in data[config["jp_col"]]:
    JP_lang.addSentence(s)

## input の加工

In [10]:
if config["src"]=="jp":
    src_col=config["jp_col"]
    trg_col=config["en_col"]
    Langs={"src":JP_lang,"trg":EN_lang}
else:
    src_col=config["en_col"]
    trg_col=config["jp_col"]
    Langs={"trg":JP_lang,"src":EN_lang}

In [11]:
input_en = data[src_col]

In [12]:
input_source_lang=data[src_col].apply(lambda x:Langs["src"].word2id(x))
input_target_lang=data[trg_col].apply(lambda x:Langs["trg"].word2id(x))

In [13]:
input_source_padded=sequence.pad_sequences(input_source_lang,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")
input_target_padded=sequence.pad_sequences(input_target_lang,maxlen=config["MAX_LENGTH"],padding="post",truncating="post")

In [14]:
input_source_padded[10]

array([50, 56, 57, 42, 58, 59, 60, 27, 61,  5,  2,  0,  0,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

In [15]:
input_target_padded[10]

array([31, 22, 62, 39, 26, 63, 64,  9, 65, 11, 66,  5,  2,  0,  0,  0,  0,
        0,  0,  0], dtype=int32)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [17]:
trainer = Seq2Seq(config)

In [18]:
trainer.trainIters(input_source_padded,input_target_padded)

epoch1 start


KeyboardInterrupt: 

# test

In [ ]:
val_data_id = val_data[src_col].apply(lambda x:Langs["src"].word2id(x))

In [ ]:
val_data_id

In [ ]:
ret=[]
for s in val_data_id:
    ret.append(trainer.translate(s))

In [ ]:
for src,pred,target in zip(val_data[src_col],ret,val_data[trg_col]):
    print("src->",src)
    print()
    print("pred->"," ".join(Langs["trg"].id2word(pred)))
    print("ans->",target)
    print("------------------")